In [64]:
import pickle
import numpy as np

In [53]:
def save_data(filename, data):
    #Storing data with labels
    a_file = open(filename, "wb")
    pickle.dump(data, a_file)
    a_file.close()
    

def load_data(filename):
    a_file = open(filename, "rb")
    output = pickle.load(a_file)
    a_file.close()
    return output

In [57]:
f = open('train_14k_split_conll.txt', 'r')
sentiment_dataset = []

start = False
for i, line in enumerate(f):
    line = line.split('\t')
    
    if len(line) == 3:
        start = True
        tweet = ''
        tweet_id = int(line[1])
        label = line[2][:-1]
    elif len(line) == 1:
        start = False
        tweet = tweet[:-1]
        sentiment_dataset.append((tweet_id, tweet, label))
    
    if start and len(line) != 3:
        if line[0] == '@' or line[0] == '#':
            tweet += line[0]
        else:
            tweet += line[0] + ' '

In [ ]:
save_data('sentiment_data_train.pkl', sentiment_dataset)

In [59]:
###removing links present after long tweets ....

In [77]:
too_long_processed_dataset = []
for i, (tweet_id, tweet, label) in enumerate(sentiment_dataset):
    tweet = tweet.split()
    check = [(ord(l),j) for j,l in enumerate(tweet) if len(l) == 1]
    
    #check if ... is in the tweet
    if len(check) > 0:
        if 8230 in np.array(check)[:,0]:
            index = np.where(np.array(check)[:,0] == 8230)[0][0]
            too_long_index = check[index][1]

            if len(tweet) > too_long_index + 1 and tweet[too_long_index+1] == 'https':
                tweet = tweet[:too_long_index + 1]

    #combine tweet into string and add to dataset      
    processed_tweet = ''
    for l in tweet:
        processed_tweet += l + ' '
    processed_tweet = processed_tweet[:-1]

    too_long_processed_dataset.append((tweet_id, processed_tweet, label))

In [70]:
save_data('sentiment_data_train.pkl', too_long_processed_dataset)

In [105]:
## Combining other links in the dataset

In [107]:
counter = 0
for i, (tweet_id, tweet, label) in enumerate(too_long_processed_dataset):
    tweet = tweet.split()
    if 'https' in tweet:
        counter += 1
        
print(counter, counter/14000)## 12% tweets still have lnks

1672 0.11942857142857143


In [113]:
handle_links = []
for i, (tweet_id, tweet, label) in enumerate(too_long_processed_dataset):
    tweet = tweet.split()

    if 'https' in tweet:
        index = tweet.index('https')
        
        link = ''
        if '.' in tweet[index:index + 7]:
            link_len = 7
        else:
            link_len = 6
        
        try:
            for i in range(link_len):
                link += tweet[index]
                tweet.pop(index)
        except:
            tweet.insert(index, link)
            
        tweet.insert(index, link)
    
    #combine tweet into string and add to dataset
    processed_tweet = ''
    for l in tweet:
        processed_tweet += l + ' '
    processed_tweet = processed_tweet[:-1]
    
    handle_links.append((tweet_id, processed_tweet, label))

In [114]:
save_data('sentiment_data_train.pkl', handle_links)